In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

data = load_boston()
data.keys()

X, y = data["data"], data["target"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

pipe = Pipeline([('std_scl',StandardScaler())])
X_train = pipe.fit_transform(X_train)
X_test = pipe.transform(X_test)

X_train = pd.DataFrame(X_train,columns=data["feature_names"])
X_test = pd.DataFrame(X_test,columns=data["feature_names"])
y_train = pd.DataFrame(y_train,columns=["target"])
y_test = pd.DataFrame(y_test,columns=["target"])

In [2]:
from sklearn.ensemble import BaggingRegressor

# Performs voting regressor aggregation
bag_reg = BaggingRegressor(base_estimator=DecisionTreeRegressor(max_depth=3,
                                                      max_features='sqrt',
                                                      splitter='random'),
                            n_estimators=30,
                            max_samples=0.8,  # int/float, default with replacement
                                              # maximum training set sample size compared to original training set 
                            max_features=0.9, # int/float, default without replacement
                                              # maximum feature size compared to original number of features 
                            bootstrap=True,   # bagging (True)/pasting (False) (affects max_samples behaviour)
                            bootstrap_features=False, # if feature selection should use baggin (max_features)
                            oob_score=False,  # perform oob scoring
                            warm_start=False,
                            n_jobs=2,
                            random_state=0,
                            verbose=0
                            )

bag_reg.fit(X_train, y_train)

/Users/maxschallwig/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/_bagging.py:399: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return column_or_1d(y, warn=True)


BaggingRegressor(base_estimator=DecisionTreeRegressor(ccp_alpha=0.0,
                                                      criterion='mse',
                                                      max_depth=3,
                                                      max_features='sqrt',
                                                      max_leaf_nodes=None,
                                                      min_impurity_decrease=0.0,
                                                      min_impurity_split=None,
                                                      min_samples_leaf=1,
                                                      min_samples_split=2,
                                                      min_weight_fraction_leaf=0.0,
                                                      presort='deprecated',
                                                      random_state=None,
                                                      splitter='random'),
                 bootstrap=

In [3]:
print(len(bag_reg.estimators_)) # array of trained estimators
print(len(bag_reg.estimators_samples_)) # array of sample subsets for each estimator
print(len(bag_reg.estimators_features_)) # array of feature subsets for each estimator

# #when oob_score is true
# bag_clf.oob_score_
# bag_clf.oob_prediction_

30
30
30


In [4]:
scores = []
for est,features in zip(bag_reg.estimators_,bag_reg.estimators_features_):
    scores.append(est.score(X_test.values[:, features], y_test))

In [5]:
print('Avg. estimator performance:',np.mean(scores))
print('Estimaor performance std. dev.:',np.std(scores))

Avg. estimator performance: 0.1779633696481234
Estimaor performance std. dev.: 0.18022204091740218


In [6]:
bag_reg.score(X_test, y_test)

0.38178205262816756